## 05 - Otimização de Hiperparâmetros e Predições Finais
1. Otimização de hiperparâmetros (Grid Search)
2. Treinamento do modelo otimizado
3. Predições no conjunto de validação
4. Comparação com modelos base

# Imports

In [ ]:
import sys
sys.path.append('/home/jovyan/work')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from src import S3Client, ModelTrainer, ModelEvaluator, MLFlowClient, DatabaseClient

# Carregamento dos dados

In [ ]:
s3 = S3Client()

X_train = s3.read_csv('processed/X_train_scaled.csv')
y_train = s3.read_csv('processed/y_train.csv')['target']
X_test = s3.read_csv('processed/X_test_scaled.csv')
y_test = s3.read_csv('processed/y_test.csv')['target']
X_val = s3.read_csv('processed/X_val_scaled.csv')
y_val = s3.read_csv('processed/y_val.csv')['target']

print(f"📊 Train: {X_train.shape}")
print(f"📊 Test: {X_test.shape}")
print(f"📊 Validation: {X_val.shape}")

# Inicialização

In [ ]:
trainer = ModelTrainer()
evaluator = ModelEvaluator()
mlflow_client = MLFlowClient(experiment_name="heart-disease-hyperparameter-tuning")

## Parte 1: Otimização de Hiperparâmetros

### Random Forest - Grid Search

In [ ]:
print("🔄 Otimizando Random Forest...")

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Modelo base
rf_base = trainer.get_model('random_forest')

# Grid Search
print(f"📊 Testando {np.prod([len(v) for v in rf_param_grid.values()])} combinações...")
print(f"⏱️  Isso pode levar alguns minutos...\n")

rf_tuned, rf_best_params = trainer.hyperparameter_tuning(
    model=rf_base,
    param_grid=rf_param_grid,
    X_train=X_train,
    y_train=y_train,
    cv=5,
    scoring='accuracy'
)

print(f"\n🏆 Melhores hiperparâmetros Random Forest:")
for param, value in rf_best_params.items():
    print(f"   {param}: {value}")

### Logistic Regression - Grid Search

In [ ]:
print("\n🔄 Otimizando Logistic Regression...")

lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000]
}

lr_base = trainer.get_model('logistic_regression')

print(f"📊 Testando {np.prod([len(v) for v in lr_param_grid.values()])} combinações...")

lr_tuned, lr_best_params = trainer.hyperparameter_tuning(
    model=lr_base,
    param_grid=lr_param_grid,
    X_train=X_train,
    y_train=y_train,
    cv=5,
    scoring='accuracy'
)

print(f"\n🏆 Melhores hiperparâmetros Logistic Regression:")
for param, value in lr_best_params.items():
    print(f"   {param}: {value}")

### SVM - Grid Search

In [ ]:
print("\n🔄 Otimizando SVM...")

svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'poly']
}

svm_base = trainer.get_model('svm')

print(f"📊 Testando {np.prod([len(v) for v in svm_param_grid.values()])} combinações...")
print("⚠️  SVM pode demorar mais...")

svm_tuned, svm_best_params = trainer.hyperparameter_tuning(
    model=svm_base,
    param_grid=svm_param_grid,
    X_train=X_train,
    y_train=y_train,
    cv=5,
    scoring='accuracy'
)

print(f"\n🏆 Melhores hiperparâmetros SVM:")
for param, value in svm_best_params.items():
    print(f"   {param}: {value}")

## Parte 2: Avaliar Modelos Otimizados

In [ ]:
tuned_models = {
    'random_forest_optimized': rf_tuned,
    'logistic_regression_optimized': lr_tuned,
    'svm_optimized': svm_tuned
}

results_tuned = []

for name, model in tuned_models.items():
    print(f"\n🔄 Avaliando: {name}")
    
    # Predições
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
    
    # Avaliar
    metrics = evaluator.evaluate_model(y_test, y_pred, y_pred_proba, model_name=name)
    results_tuned.append(metrics)
    
    # Registrar no MLFlow
    run = mlflow_client.start_run(run_name=f"tuned_{name}")
    
    mlflow_client.log_params(model.get_params())
    mlflow_client.log_metrics({
        'test_accuracy': metrics['accuracy'],
        'test_precision': metrics['precision'],
        'test_recall': metrics['recall'],
        'test_f1': metrics['f1_score']
    })
    mlflow_client.log_model(model, artifact_path="model", registered_model_name=name)
    mlflow_client.end_run()

print("\n✅ Modelos otimizados avaliados!")

### Comparação: Base vs Otimizado

In [ ]:
base_models_names = ['random_forest', 'logistic_regression', 'svm']
results_base = []

for name in base_models_names:
    try:
        # Carregar modelo base do MLFlow
        model_uri = f"models:/{name}/latest"
        model = mlflow_client.load_model(model_uri)
        
        # Predições
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None
        
        # Avaliar
        metrics = evaluator.evaluate_model(y_test, y_pred, y_pred_proba, model_name=name)
        results_base.append(metrics)
    except:
        print(f"⚠️  Modelo base {name} não encontrado")

In [ ]:
comparison_data = []

for base, tuned in zip(results_base, results_tuned):
    comparison_data.append({
        'Model': base['model_name'].replace('_', ' ').title(),
        'Base Accuracy': base['accuracy'],
        'Tuned Accuracy': tuned['accuracy'],
        'Improvement': tuned['accuracy'] - base['accuracy'],
        'Improvement %': (tuned['accuracy'] - base['accuracy']) / base['accuracy'] * 100
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n📊 Comparação Base vs Otimizado:")
display(comparison_df)

### Visualizações

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Base Accuracy'], width, 
               label='Base', color='#3498db', alpha=0.8)
bars2 = ax.bar(x + width/2, comparison_df['Tuned Accuracy'], width,
               label='Otimizado', color='#e74c3c', alpha=0.8)

ax.set_xlabel('Modelo')
ax.set_ylabel('Accuracy')
ax.set_title('Comparação: Modelos Base vs Otimizados')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Model'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## Parte 3: Predições no Conjunto de Validação

In [ ]:
best_tuned = max(results_tuned, key=lambda x: x['accuracy'])
best_model_name = best_tuned['model_name']
best_accuracy = best_tuned['accuracy']

print(f"\n🏆 Melhor modelo otimizado: {best_model_name}")
print(f"   Accuracy: {best_accuracy:.4f}")

# Carregar melhor modelo
best_model = tuned_models[best_model_name]

In [ ]:
print("\n🔄 Realizando predições no conjunto de validação...")

y_val_pred = best_model.predict(X_val)
y_val_pred_proba = best_model.predict_proba(X_val) if hasattr(best_model, 'predict_proba') else None

# Avaliar
val_metrics = evaluator.evaluate_model(
    y_val, 
    y_val_pred, 
    y_val_pred_proba,
    model_name=f"{best_model_name}_validation"
)

print(f"\n📊 Resultados no Conjunto de Validação:")
print(f"   Accuracy:  {val_metrics['accuracy']:.4f}")
print(f"   Precision: {val_metrics['precision']:.4f}")
print(f"   Recall:    {val_metrics['recall']:.4f}")
print(f"   F1-Score:  {val_metrics['f1_score']:.4f}")

In [ ]:
cm_val = evaluator.get_confusion_matrix(y_val, y_val_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Saudável', 'Doença'],
           yticklabels=['Saudável', 'Doença'])
plt.title(f'Matriz de Confusão - Validação\n{best_model_name}')
plt.ylabel('Real')
plt.xlabel('Predito')
plt.tight_layout()
plt.show()

# Salvar predições e Modelo Final

In [ ]:
predictions_df = pd.DataFrame({
    'patient_id': range(len(y_val)),
    'true_label': y_val.values,
    'predicted_label': y_val_pred,
    'probability_disease': y_val_pred_proba[:, 1] if y_val_pred_proba is not None else y_val_pred,
    'correct': y_val.values == y_val_pred
})

print("\n📊 Primeiras predições:")
display(predictions_df.head(10))

# Estatísticas
correct_count = predictions_df['correct'].sum()
total_count = len(predictions_df)

print(f"\n✅ Predições corretas: {correct_count}/{total_count} ({correct_count/total_count*100:.1f}%)")

In [ ]:
s3.write_csv(predictions_df, 'predictions/validation_predictions.csv')
print("\n✅ Predições salvas no MinIO!")

In [ ]:
# Registrar como modelo de produção
model_version = mlflow_client.register_model(
    run_id=mlflow_client.client.search_runs(
        experiment_ids=[mlflow_client.experiment_id],
        filter_string=f"tags.mlflow.runName = 'tuned_{best_model_name}'",
        max_results=1
    )[0].info.run_id,
    model_name="production_model",
    artifact_path="model"
)

# Transicionar para produção
mlflow_client.transition_model_stage(
    model_name="production_model",
    version=model_version,
    stage="Production"
)

print(f"\n✅ Modelo de produção registrado: production_model v{model_version}")

# Resumo Final

print("\n" + "="*60)
print("📊 RESUMO FINAL DO PROJETO")
print("="*60)

print("\n🎯 MODELOS BASE (Artigo):")
for result in results_base:
    print(f"   {result['model_name']}: {result['accuracy']:.4f}")

print("\n🚀 MODELOS OTIMIZADOS:")
for result in results_tuned:
    print(f"   {result['model_name']}: {result['accuracy']:.4f}")

print(f"\n🏆 MELHOR MODELO: {best_model_name}")
print(f"   Test Accuracy:       {best_accuracy:.4f}")
print(f"   Validation Accuracy: {val_metrics['accuracy']:.4f}")

print(f"\n📈 MELHORIA MÉDIA: {comparison_df['Improvement %'].mean():.2f}%")

print("\n✅ PROJETO CONCLUÍDO!")
print("="*60)